In [283]:
library(ppcor)
#install.packages('lmPerm')
library(lmPerm)
library(psych)
library(car)
library(ggplot2)
library(lme4)
library(mediation)
library(caret)
library(gbm)
library(party)
library(Metrics)
#install.packages("wesanderson")
library(wesanderson)
library(gtools)
library(interactions)
#install.packages('ggstance')
library(ggstance)
#install.packages('elasticnet')
library(elasticnet)
#install.packages('kernlab')
library(kernlab)
#install.packages('e1071')
library(e1071)
#install.packages('MatchIt')
library(MatchIt)
library(dplyr)
library(ukbtools)
library(mediation)
library(lavaan)
library(lavaanPlot)

# Load data and exclude participants
Exclusion criteria:
- Bipolar disorder
- Personality disorders
- Schizophrenia
- Depression
- Mania
- Bulimia nervosa
- Anorexia nervosa
- ADD/ADHD
- Panic attacks
- Diabetes
- Hypertension
- Neurological disorders
- Underweight (BMI<18.5)

In [1513]:
my_ukb_data <- ukb_df("ukb38276", path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
load("/home/bic/fmorys/Projects/UKBB_all/di.Rda")
colnames(di)=c('eid',colnames(di)[2:length(di)])
ukb_area <- ukb_df('ukb40807', path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
glob2rx("area_of*_2_*") # Create a regexp to use in grep
ukbb_area=(ukb_area[c(1, grep('^area_of.*_2_',colnames(ukb_area)))])

### Exclude outliers in the imaging dataset (n=22k)
di$bipolar_dis <- NA
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type I (Mania)"] <- 1
di$bipolar_dis[di$bipolar_disorder_status_f20122_0_0=="Bipolar Type II (Hypomania)"] <- 1

#Personality Disorders
di$personality_dis <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$personality_dis[di[i]=="A personality disorder"] <- 1
}

#Mental health issues
di$schizophrenia <- NA
for (i in as.numeric(grep('mental_health_problems_ever_diagnosed_by_a_professional', 
                          colnames(di)))) { #Columns with mental illness
di$schizophrenia[di[i]=="Schizophrenia"] <- 1
di$schizophrenia[di[i]=="Depression"] <- 1
di$schizophrenia[di[i]=="Mania, hypomania, bipolar or manic-depression"] <- 1
di$schizophrenia[di[i]=="Bulimia nervosa"] <- 1
di$schizophrenia[di[i]=="Anorexia nervosa"] <- 1
di$schizophrenia[di[i]=="Attention deficit or attention deficit and hyperactivity disorder (ADD/ADHD)"] <- 1
di$schizophrenia[di[i]=="Panic attacks"] <- 1
}


# Exclude participants who have diabetes
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('diabetes_diagnosed_by_doctor', 
                          colnames(di)))) { #Columns with vascular diseases
di$diabetes[di[i] =="Yes"] <- 1
}

# Exclude participants who had a heart attack, angina or stroke
di$vascular_heart_diagnoses <- NA
for (i in as.numeric(grep('vascularheart_problems_diagnosed_by_doctor', 
                          colnames(di)))) { #Columns with vascular diseases
di$vascular_heart_diagnoses[di[i] =="High blood pressure"] <- 1
}



#Dementia
for (i in as.numeric(grep('noncancer_illness_code_selfreported', 
                          colnames(di)))) { #Columns with noncancer illness for all timepoints
di$neurological_disorder[di[i] == "1263"] <- 1 #Dementia or Alzheimer's
di$neurological_disorder[di[i] == "1262"] <- 1 #Parkinson Disease
di$neurological_disorder[di[i] == "1258"] <- 1 #Chronic degenerative neurological
di$neurological_disorder[di[i] == "1256"] <- 1 #Guillain Barré
di$neurological_disorder[di[i] == "1261"] <- 1 #Multiple slerosis
di$neurological_disorder[di[i] == "1297"] <- 1 #Other demyenilating disease
di$neurological_disorder[di[i] == "1081"] <- 1 #Stroke
di$neurological_disorder[di[i] == "1032"] <- 1 #Brain cancer
di$neurological_disorder[di[i] == "1491"] <- 1 #Brain hemorrhage
di$neurological_disorder[di[i] == "1245"] <- 1 #Brain/ intracranial abcess
di$neurological_disorder[di[i] == "1425"] <- 1 #Cerebral aneurism
di$neurological_disorder[di[i] == "1433"] <- 1 #Cerebral palsy
di$neurological_disorder[di[i] == "1256"] <- 1 #Encephalitis
di$neurological_disorder[di[i] == "1264"] <- 1 #Epilepsy
di$neurological_disorder[di[i] == "1266"] <- 1 #Head injury
di$neurological_disorder[di[i] == "1244"] <- 1 #Infections of the nervous system
di$neurological_disorder[di[i] == "1583"] <- 1 #Ischaemic stroke
di$neurological_disorder[di[i] == "1031"] <- 1 #Meningeal cancer
di$neurological_disorder[di[i] == "1659"] <- 1 #Meningioma (benign)
di$neurological_disorder[di[i] == "1247"] <- 1 #Meningitis
di$neurological_disorder[di[i] == "1259"] <- 1 #Motor neuron disease
di$neurological_disorder[di[i] == "1240"] <- 1 #Neurological injury/trauma
di$neurological_disorder[di[i] == "1524"] <- 1 #Spina bifida
di$neurological_disorder[di[i] == "1083"] <- 1 #Subdural hematoma
di$neurological_disorder[di[i] == "1086"] <- 1 #Subarachnoid haemorrage
di$neurological_disorder[di[i] == "1082"] <- 1 #Transient ischaemic attack
}

#If BMI < 18.5 (Re-do this, cause it's not usually done)
#di$underweight1[di$body_mass_index_bmi_f21001_0_0 < 18.5] <- 1
#di$underweight2[di$body_mass_index_bmi_f21001_1_0 < 18.5] <- 1
di$underweight3[di$body_mass_index_bmi_f21001_2_0 < 18.5] <- 1

#Exclude participants if no neuroimaging data, bipolar_dis == 1; personality_dis == 1; schizophrenia == 1; vascular_heart_diagnoses ==1

di$excluded <- NA
di$excluded[di$bipolar_dis ==1] <- 1
di$excluded[di$personality_dis == 1] <- 1
di$excluded[di$schizophrenia == 1] <- 1
di$excluded[di$vascular_heart_diagnoses ==1] <- 1
#di$excluded[di$underweight1 ==1] <- 1
#di$excluded[di$underweight2 ==1] <- 1
di$excluded[di$underweight3 == 1] <- 1
di$excluded[di$neurological_disorder ==1] <- 1
di$excluded[di$diabetes ==1] <- 1

di$included<-car::recode(di$excluded, "1='excluded'; else='included'")


#Select subjects if included ==1
di_excluded <- subset(di, included=="included")

ukbb_all=merge(my_ukb_data, di_excluded, by.x='eid')
ukbb_all=merge(ukbb_all, ukbb_area, by='eid',all.x=TRUE)

nrow(ukbb_all)

ukbb_all$WMH_logscaled=log(ukbb_all$total_volume_of_white_matter_hyperintensities_from_t1_and_t2_flair_images_f25781_2_0*
                    ukbb_all$volumetric_scaling_from_t1_head_image_to_standard_space_f25000_2_0)
ukbb_all$logBMI3=log(ukbb_all$body_mass_index_bmi_f21001_2_0)
ukbb_all$WHR=ukbb_all$waist_circumference_f48_2_0/ukbb_all$hip_circumference_f49_2_0

## Merge with white matter integrity variables
ukb_WM <- ukb_df('ukb35375', path = "/dagher/dagher11/filip/OBAL/UKBB_data/")
ukbb_all=merge(ukbb_all,ukb_WM,by='eid',all.x=TRUE)

#Remove outliers
variables_for_outliers=c(58:88, 120:150, 26:27, 32:38, 42:43, 48:54, 89:119, 
                         151:181, 9, 2287:2300, 2116, 5094:5096, 5123:5218,
                         grep('^area_of.*_2_',colnames(ukbb_all))) # all volume and thickness variables from FS, BMI, eTIV, area

for (j in variables_for_outliers) { # 
    Q3=as.numeric(quantile(ukbb_all[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(ukbb_all[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    ukbb_all[j][ukbb_all[j]<lower]=NA
    ukbb_all[j][ukbb_all[j]>upper]=NA
}

ukbb_all=ukbb_all[complete.cases(ukbb_all[,5094:5096]),] # BMI, WMH, WHR
nrow(ukbb_all)


# Creating sets of data for each imaging feature:
- Cortical thickness
- Cortical volume
- Subcortical volume as calculated by FSL
- Surface area

In [1521]:
ukbb_ct=ukbb_all[complete.cases(ukbb_all[,c(58:88, 120:150)]),] # CT
ukbb_ctxvol=ukbb_all[complete.cases(ukbb_all[,c(89:119, 151:181, 9)]),]
ukbb_subvol=ukbb_all[complete.cases(ukbb_all[,c(2287:2300, 2116)]),]
ukbb_area=ukbb_all[complete.cases(ukbb_all[,c(grep('^area_of.*_2_',colnames(ukbb_all)))]),]
WM=ukbb_WM[complete.cases(ukbb_WM[,c(5123:5218)]),]

# Data analysis
# BMI -> WMH -> brain measures

# BMI -> WMH

In [1211]:
set.seed(2014)
med.fit <- lm(scale(ukbb_ct$WMH_logscaled) ~
          scale(ukbb_ct$logBMI3) +
          scale(ukbb_ct$age_when_attended_assessment_centre_f21003_2_0) +
          ukbb_ct$sex_f31_0_0, data=ukbb_ct)

## WMH -> CT

In [1189]:
Stats=matrix(nrow=length(c(58:88, 120:150)),ncol=10) # CT variables
index=1

for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
  
    out.fit <- lm(scale(ukbb_ct[[i]]) ~ 
                scale(ukbb_ct$WMH_logscaled) +
                scale(ukbb_ct$logBMI3) +
                scale(ukbb_ct$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_ct$sex_f31_0_0, data=ukbb_ct)
    
    med.out <- mediate(med.fit, out.fit, treat='scale(ukbb_ct$logBMI3)',
                   mediator='scale(ukbb_ct$WMH_logscaled)',
                   robustSE=TRUE,
                   sims=1000, data=ukbb_ct)
    
    med2dep <- summary(lm(scale(ukbb_ct[[i]]) ~ 
                scale(ukbb_ct$WMH_logscaled) +
                scale(ukbb_ct$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_ct$sex_f31_0_0, data=ukbb_ct)) 
      
    
    Stats[index,][1]=colnames(ukbb_ct)[i]
    Stats[index,][2]=med.out$d0 # ACME estimate
    Stats[index,][3]=med.out$d0.p # ACME p-value
    Stats[index,][4]=med.out$z0 # ADE e
    Stats[index,][5]=med.out$z0.p # ADE p
    Stats[index,][6]=med.out$tau.coef # Total e
    Stats[index,][7]=med.out$tau.p # Total p
    Stats[index,][8]=med.out$n0 # Proportion mediated e
    Stats[index,][9]=med.out$n0.p # Proportion mediated p
    Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> CT
       
     
    index=index+1
}
Stats_adj=Stats
Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

In [1190]:
write.table(Stats_adj[,2:10],'/dagher/dagher11/filip/OBAL/output/WMH_CT_mediation.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table(Stats_adj[,1:10],'/dagher/dagher11/filip/OBAL/output/WMH_CT_mediation_regnames.csv',sep=',',quote=FALSE, row.names=FALSE)

## WMH -> cortical volume

In [1212]:
set.seed(2014)
med.fit <- lm(scale(ukbb_ctxvol$WMH_logscaled) ~
          scale(ukbb_ctxvol$logBMI3) +
          scale(ukbb_ctxvol$age_when_attended_assessment_centre_f21003_2_0) +
          ukbb_ctxvol$sex_f31_0_0, data=ukbb_ctxvol)

In [1213]:
Stats=matrix(nrow=length(c(89:119, 151:181)),ncol=10) # volume variables
index=1

for (i in c(89:119, 151:181)) { #All volume variables 
  
    out.fit <- lm(scale(ukbb_ctxvol[[i]]/ukbb_ctxvol$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0) ~ 
                scale(ukbb_ctxvol$WMH_logscaled) +
                scale(ukbb_ctxvol$logBMI3) +
                scale(ukbb_ctxvol$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_ctxvol$sex_f31_0_0, data=ukbb_ctxvol)
    
    med.out <- mediate(med.fit, out.fit, treat='scale(ukbb_ctxvol$logBMI3)',
                   mediator='scale(ukbb_ctxvol$WMH_logscaled)',
                   robustSE=TRUE,
                   sims=1000, data=ukbb_ctxvol)

    med2dep <- summary(lm(scale(ukbb_ctxvol[[i]]/ukbb_ctxvol$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0) ~ 
                scale(ukbb_ctxvol$WMH_logscaled) +
                scale(ukbb_ctxvol$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_ctxvol$sex_f31_0_0, data=ukbb_ctxvol)) 
      
    
    Stats[index,][1]=colnames(ukbb_ctxvol)[i]
    Stats[index,][2]=med.out$d0 # ACME estimate
    Stats[index,][3]=med.out$d0.p # ACME p-value
    Stats[index,][4]=med.out$z0 # ADE e
    Stats[index,][5]=med.out$z0.p # ADE p
    Stats[index,][6]=med.out$tau.coef # Total e
    Stats[index,][7]=med.out$tau.p # Total p
    Stats[index,][8]=med.out$n0 # Proportion mediated e
    Stats[index,][9]=med.out$n0.p # Proportion mediated p
    Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol
       
     
    index=index+1
    
}

Stats_adj=Stats
Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

In [1214]:
write.table(Stats_adj[,2:10],'/dagher/dagher11/filip/OBAL/output/WMH_VOL_mediation.csv',sep=',',quote=FALSE, row.names=FALSE)

## WMH -> Subcortical volumes

In [1215]:
set.seed(2014)
med.fit <- lm(scale(ukbb_subvol$WMH_logscaled) ~
          scale(ukbb_subvol$logBMI3) +
          scale(ukbb_subvol$age_when_attended_assessment_centre_f21003_2_0) +
          ukbb_subvol$sex_f31_0_0, data=ukbb_subvol)

In [1216]:
Stats=matrix(nrow=length(c(2287:2300)),ncol=10) # volume variables
index=1

for (i in c(2287:2300)) { #All volume variables 
  
    out.fit <- lm(scale(ukbb_subvol[[i]]*ukbb_subvol[[2116]]) ~ 
                scale(ukbb_subvol$WMH_logscaled) +
                scale(ukbb_subvol$logBMI3) +
                scale(ukbb_subvol$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_subvol$sex_f31_0_0, data=ukbb_subvol)
    
    med.out <- mediate(med.fit, out.fit, treat='scale(ukbb_subvol$logBMI3)',
                   mediator='scale(ukbb_subvol$WMH_logscaled)',
                   robustSE=TRUE,
                   sims=1000, data=ukbb_subvol)

    med2dep <- summary(lm(scale(ukbb_subvol[[i]]*ukbb_subvol[[2116]]) ~ 
                scale(ukbb_subvol$WMH_logscaled) +
                scale(ukbb_subvol$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_subvol$sex_f31_0_0, data=ukbb_subvol)) 
      
    
    Stats[index,][1]=colnames(ukbb_subvol)[i]
    Stats[index,][2]=med.out$d0 # ACME estimate
    Stats[index,][3]=med.out$d0.p # ACME p-value
    Stats[index,][4]=med.out$z0 # ADE e
    Stats[index,][5]=med.out$z0.p # ADE p
    Stats[index,][6]=med.out$tau.coef # Total e
    Stats[index,][7]=med.out$tau.p # Total p
    Stats[index,][8]=med.out$n0 # Proportion mediated e
    Stats[index,][9]=med.out$n0.p # Proportion mediated p
    Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol
       
     
    index=index+1
    
}

Stats_adj=Stats
Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

In [1217]:
write.table(Stats_adj[,1:10],'/dagher/dagher11/filip/OBAL/output/WMH_subVOL_mediationFSL.csv',sep=',',quote=FALSE, row.names=FALSE)

In [1218]:
Stats_adj=read.table('/dagher/dagher11/filip/OBAL/output/WMH_subVOL_mediationFSL.csv',sep=',',header=TRUE)
Stats_sum=Stats_adj[Stats_adj[3]<0.05 & Stats_adj[7]<0.05 & Stats_adj[10]<0.05 & Stats_adj[6]<0,]
Stats_sum

# Cognitive measures

## BMI relationship with cognitive factors

Transform variables and create a dataset with cognitive variables

In [1319]:
ukbb_all$Mean_pairs=rowMeans(ukbb_all[274:276], na.rm=TRUE, dims=1)
ukbb_all$RT=rowMeans(ukbb_all[418:429], na.rm=TRUE, dims=1)
ukbb_all$Prospective_memory<-as.numeric(car::recode(ukbb_all$prospective_memory_result_f20018_2_0, 
                                         "'Correct recall on first attempt'='1'; 
                                        'Correct recall on second attempt'='2';
                                        'Instruction not recalled, either skipped or incorrect'='3'"))
ukbb_cog=ukbb_all

#ukbb_cog=ukbb_cog[complete.cases(ukbb_cog[,c(942, 1702, 4537, 5034, 5035, 5033)]),]


## Exclude cognitive outliers

In [1320]:
variables_for_outliers=c(942, 1702, 4537, 5098, 5097) # all cognitive variable but PM because it's categorical

for (j in variables_for_outliers) { # 
    Q3=as.numeric(quantile(ukbb_cog[j],0.75, na.rm=TRUE))
    Q1=as.numeric(quantile(ukbb_cog[j],0.25, na.rm=TRUE))
    upper=Q3+(2.2*(Q3-Q1))
    lower=Q1-(2.2*(Q3-Q1))
    ukbb_cog[j][ukbb_cog[j]<lower]=NA
    ukbb_cog[j][ukbb_cog[j]>upper]=NA
}

## Investigate relationship between BMI and cognitive measures as a prerequisite for mediation

In [1221]:
var_list=c('maximum_digits_remembered_correctly_f4282_2_0',
          'fluid_intelligence_score_f20016_2_0',
          'number_of_puzzles_correct_f21004_2_0',
          'Prospective_memory',
          'Mean_pairs',
          'RT')


for (i in 1:length(var_list)) {
    
    
    ukbb_cogloop=ukbb_cog[complete.cases(ukbb_cog[,grep(var_list[i], colnames(ukbb_cog))]),]
    
    
    print(summary(lm((as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~
              scale(ukbb_cogloop$logBMI3) +
              scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
              ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)))
    }

## BMI -> WMH -> cognitive measures
Only for significant measures:
- Digit span
- Fluid intelligence
- Prospective memory
- Visual memory

In [1222]:
var_list=c('maximum_digits_remembered_correctly_f4282_2_0',
          'fluid_intelligence_score_f20016_2_0',
          'Prospective_memory',
          'Mean_pairs')

Stats=matrix(nrow=length(var_list),ncol=10) 
index=1
set.seed(2014)

ukbb_all$Mean_pairs=rowMeans(ukbb_all[274:276], na.rm=TRUE, dims=1)
ukbb_all$RT=rowMeans(ukbb_all[418:429], na.rm=TRUE, dims=1)
ukbb_all$Prospective_memory<-as.numeric(car::recode(ukbb_all$prospective_memory_result_f20018_2_0, 
                                         "'Correct recall on first attempt'='1'; 
                                        'Correct recall on second attempt'='2';
                                        'Instruction not recalled, either skipped or incorrect'='3'"))


for (i in 1:length(var_list)) {
    
    ukbb_cogloop=ukbb_cog[complete.cases(ukbb_cog[,grep(var_list[i], colnames(ukbb_cog))]),]
    

    med.fit <- lm(scale(ukbb_cogloop$WMH_logscaled) ~
              scale(ukbb_cogloop$logBMI3) +
              scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
              ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)
    
    out.fit_cog=lm((as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~
              scale(ukbb_cogloop$WMH_logscaled) +
              scale(ukbb_cogloop$logBMI3) +
              scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
              ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)
    
    
    med.out <- mediate(med.fit, out.fit_cog, treat='scale(ukbb_cogloop$logBMI3)',
                   mediator='scale(ukbb_cogloop$WMH_logscaled)',
                   robustSE=TRUE,
                   sims=1000, data=ukbb_cogloop)
    
    med2dep <- summary(lm(scale(as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~ 
            scale(ukbb_cogloop$WMH_logscaled) +
            scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
            ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop))
    
    Stats[index,][1]=var_list[i]
    Stats[index,][2]=med.out$d0 # ACME estimate
    Stats[index,][3]=med.out$d0.p # ACME p-value
    Stats[index,][4]=med.out$z0 # ADE e
    Stats[index,][5]=med.out$z0.p # ADE p
    Stats[index,][6]=med.out$tau.coef # Total e
    Stats[index,][7]=med.out$tau.p # Total p
    Stats[index,][8]=med.out$n0 # Proportion mediated e
    Stats[index,][9]=med.out$n0.p # Proportion mediated p
    Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol
       
     
    index=index+1
    
    }      
    
Stats_adj=Stats
Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('Cognitive test','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

In [1223]:
Stats_adj

# WHR -> WMH -> brain measures

## WHR -> WMH

In [1224]:
set.seed(2014)
med.fit <- lm(scale(ukbb_ct$WMH_logscaled) ~
          scale(ukbb_ct$WHR) +
          scale(ukbb_ct$age_when_attended_assessment_centre_f21003_2_0) +
          ukbb_ct$sex_f31_0_0, data=ukbb_ct)

## WMH -> CT

In [1225]:
Stats=matrix(nrow=length(c(58:88, 120:150)),ncol=10) # CT variables
index=1

for (i in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
  
    out.fit <- lm(scale(ukbb_ct[[i]]) ~ 
                scale(ukbb_ct$WMH_logscaled) +
                scale(ukbb_ct$WHR) +
                scale(ukbb_ct$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_ct$sex_f31_0_0, data=ukbb_ct)
    
    med.out <- mediate(med.fit, out.fit, treat='scale(ukbb_ct$WHR)',
                   mediator='scale(ukbb_ct$WMH_logscaled)',
                   robustSE=TRUE,
                   sims=1000, data=ukbb_ct)
    
    med2dep <- summary(lm(scale(ukbb_ct[[i]]) ~ 
                scale(ukbb_ct$WMH_logscaled) +
                scale(ukbb_ct$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_ct$sex_f31_0_0, data=ukbb_ct)) 
      
    
    Stats[index,][1]=colnames(ukbb_ct)[i]
    Stats[index,][2]=med.out$d0 # ACME estimate
    Stats[index,][3]=med.out$d0.p # ACME p-value
    Stats[index,][4]=med.out$z0 # ADE e
    Stats[index,][5]=med.out$z0.p # ADE p
    Stats[index,][6]=med.out$tau.coef # Total e
    Stats[index,][7]=med.out$tau.p # Total p
    Stats[index,][8]=med.out$n0 # Proportion mediated e
    Stats[index,][9]=med.out$n0.p # Proportion mediated p
    Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> CT
       
     
    index=index+1
}
Stats_adj=Stats
Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

In [1226]:
write.table(Stats_adj[,2:10],'/dagher/dagher11/filip/OBAL/output/WMH_CT_mediation_WHR.csv',sep=',',quote=FALSE, row.names=FALSE)
write.table(Stats_adj[,1:10],'/dagher/dagher11/filip/OBAL/output/WMH_CT_mediation_regnames_WHR.csv',sep=',',quote=FALSE, row.names=FALSE)

## WMH -> cortical volume

In [1227]:
set.seed(2014)
med.fit <- lm(scale(ukbb_ctxvol$WMH_logscaled) ~
          scale(ukbb_ctxvol$WHR) +
          scale(ukbb_ctxvol$age_when_attended_assessment_centre_f21003_2_0) +
          ukbb_ctxvol$sex_f31_0_0, data=ukbb_ctxvol)

In [1228]:
Stats=matrix(nrow=length(c(89:119, 151:181)),ncol=10) # volume variables
index=1

for (i in c(89:119, 151:181)) { #All volume variables 
  
    out.fit <- lm(scale(ukbb_ctxvol[[i]]/ukbb_ctxvol$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0) ~ 
                scale(ukbb_ctxvol$WMH_logscaled) +
                scale(ukbb_ctxvol$WHR) +
                scale(ukbb_ctxvol$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_ctxvol$sex_f31_0_0, data=ukbb_ctxvol)
    
    med.out <- mediate(med.fit, out.fit, treat='scale(ukbb_ctxvol$WHR)',
                   mediator='scale(ukbb_ctxvol$WMH_logscaled)',
                   robustSE=TRUE,
                   sims=1000, data=ukbb_ctxvol)

    med2dep <- summary(lm(scale(ukbb_ctxvol[[i]]/ukbb_ctxvol$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0) ~ 
                scale(ukbb_ctxvol$WMH_logscaled) +
                scale(ukbb_ctxvol$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_ctxvol$sex_f31_0_0, data=ukbb_ctxvol)) 
      
    
    Stats[index,][1]=colnames(ukbb_ctxvol)[i]
    Stats[index,][2]=med.out$d0 # ACME estimate
    Stats[index,][3]=med.out$d0.p # ACME p-value
    Stats[index,][4]=med.out$z0 # ADE e
    Stats[index,][5]=med.out$z0.p # ADE p
    Stats[index,][6]=med.out$tau.coef # Total e
    Stats[index,][7]=med.out$tau.p # Total p
    Stats[index,][8]=med.out$n0 # Proportion mediated e
    Stats[index,][9]=med.out$n0.p # Proportion mediated p
    Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol
       
     
    index=index+1
    
}

Stats_adj=Stats
Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

In [1229]:
write.table(Stats_adj[,2:10],'/dagher/dagher11/filip/OBAL/output/WMH_VOL_mediation_WHR.csv',sep=',',quote=FALSE, row.names=FALSE)

## WMH -> Subcortical volumes

In [1230]:
set.seed(2014)
med.fit <- lm(scale(ukbb_subvol$WMH_logscaled) ~
          scale(ukbb_subvol$WHR) +
          scale(ukbb_subvol$age_when_attended_assessment_centre_f21003_2_0) +
          ukbb_subvol$sex_f31_0_0, data=ukbb_subvol)

In [1231]:
Stats=matrix(nrow=length(c(2287:2300)),ncol=10) # volume variables
index=1

for (i in c(2287:2300)) { #All volume variables 
  
    out.fit <- lm(scale(ukbb_subvol[[i]]*ukbb_subvol[[2116]]) ~ 
                scale(ukbb_subvol$WMH_logscaled) +
                scale(ukbb_subvol$WHR) +
                scale(ukbb_subvol$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_subvol$sex_f31_0_0, data=ukbb_subvol)
    
    med.out <- mediate(med.fit, out.fit, treat='scale(ukbb_subvol$WHR)',
                   mediator='scale(ukbb_subvol$WMH_logscaled)',
                   robustSE=TRUE,
                   sims=1000, data=ukbb_subvol)

    med2dep <- summary(lm(scale(ukbb_subvol[[i]]*ukbb_subvol[[2116]]) ~ 
                scale(ukbb_subvol$WMH_logscaled) +
                scale(ukbb_subvol$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_subvol$sex_f31_0_0, data=ukbb_subvol)) 
      
    
    Stats[index,][1]=colnames(ukbb_subvol)[i]
    Stats[index,][2]=med.out$d0 # ACME estimate
    Stats[index,][3]=med.out$d0.p # ACME p-value
    Stats[index,][4]=med.out$z0 # ADE e
    Stats[index,][5]=med.out$z0.p # ADE p
    Stats[index,][6]=med.out$tau.coef # Total e
    Stats[index,][7]=med.out$tau.p # Total p
    Stats[index,][8]=med.out$n0 # Proportion mediated e
    Stats[index,][9]=med.out$n0.p # Proportion mediated p
    Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol
       
     
    index=index+1
    
}

Stats_adj=Stats
Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

In [1232]:
write.table(Stats_adj[,1:10],'/dagher/dagher11/filip/OBAL/output/WMH_subVOL_mediation_WHRFSL.csv',sep=',',quote=FALSE, row.names=FALSE)

In [1233]:
Stats_adj=read.table('/dagher/dagher11/filip/OBAL/output/WMH_subVOL_mediation_WHRFSL.csv',sep=',',header=TRUE)
Stats_sum=Stats_adj[Stats_adj[3]<0.05 & Stats_adj[7]<0.05 & Stats_adj[10]<0.05 & Stats_adj[6]<0,]
Stats_sum

# Cognitive measures

## WHR relationship with cognitive factors
Only significant are Digit span, fluid intelligence and prospective memory

In [1234]:
var_list=c('maximum_digits_remembered_correctly_f4282_2_0',
          'fluid_intelligence_score_f20016_2_0',
          'number_of_puzzles_correct_f21004_2_0',
          'Prospective_memory',
          'Mean_pairs',
          'RT')


for (i in 1:length(var_list)) {
    
    ukbb_cogloop=ukbb_cog[complete.cases(ukbb_cog[,grep(var_list[i], colnames(ukbb_cog))]),]
    
    
    print(summary(lm((as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~
              scale(ukbb_cogloop$WHR) +
              scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
              ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)))
    }

## WHR -> WMH -> cognitive measures
Only for cognitive measures significantly related to BMI: 
- Digit span
- Fluid intelligence
- Prospective memory
- Reaction time

In [1235]:
var_list=c('maximum_digits_remembered_correctly_f4282_2_0',
          'fluid_intelligence_score_f20016_2_0',
          'Prospective_memory',
          'RT')

Stats=matrix(nrow=length(var_list),ncol=10)
index=1
set.seed(2014)




for (i in 1:length(var_list)) {
    
    ukbb_cogloop=ukbb_cog[complete.cases(ukbb_cog[,grep(var_list[i], colnames(ukbb_cog))]),]
    

    med.fit <- lm(scale(ukbb_cogloop$WMH_logscaled) ~
              scale(ukbb_cogloop$WHR) +
              scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
              ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)
    
    out.fit_cog=lm((as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~
              scale(ukbb_cogloop$WMH_logscaled) +
              scale(ukbb_cogloop$WHR) +
              scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
              ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)
    
    
    med.out <- mediate(med.fit, out.fit_cog, treat='scale(ukbb_cogloop$WHR)',
                   mediator='scale(ukbb_cogloop$WMH_logscaled)',
                   robustSE=TRUE,
                   sims=1000, data=ukbb_cogloop)
    
    med2dep <- summary(lm(scale(as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~ 
            scale(ukbb_cogloop$WMH_logscaled) +
            scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
            ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop))
    
    Stats[index,][1]=var_list[i]
    Stats[index,][2]=med.out$d0 # ACME estimate
    Stats[index,][3]=med.out$d0.p # ACME p-value
    Stats[index,][4]=med.out$z0 # ADE e
    Stats[index,][5]=med.out$z0.p # ADE p
    Stats[index,][6]=med.out$tau.coef # Total e
    Stats[index,][7]=med.out$tau.p # Total p
    Stats[index,][8]=med.out$n0 # Proportion mediated e
    Stats[index,][9]=med.out$n0.p # Proportion mediated p
    Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol
       
     
    index=index+1
    
    }      
    
Stats_adj=Stats
Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('Cognitive test','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

In [1236]:
Stats_adj

## BMI -> CT/VOL/SUBVOL -> Cognition

### CT

In [1250]:
var_list=c('maximum_digits_remembered_correctly_f4282_2_0',
          'fluid_intelligence_score_f20016_2_0',
          'Prospective_memory',
          'Mean_pairs')


for (i in 1:length(var_list)) {
    
    ukbb_cogloop=ukbb_cog[complete.cases(ukbb_cog[,grep(var_list[i], colnames(ukbb_cog))]),]
    
    Stats=matrix(nrow=length(c(58:88, 120:150)),ncol=10) # CT variables
    index=1

    for (j in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
  
        ukbb_cogloop=ukbb_cogloop[complete.cases(ukbb_cogloop[,c(58:88, 120:150)]),]

        med.fit <- lm(scale(ukbb_cogloop[[j]]) ~
                  scale(ukbb_cogloop$logBMI3) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)

        out.fit_cog=lm((as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~
                  scale(ukbb_cogloop[[j]]) +
                  scale(ukbb_cogloop$logBMI3) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)


        med.out <- mediate(med.fit, out.fit_cog, treat='scale(ukbb_cogloop$logBMI3)',
                       mediator='scale(ukbb_cogloop[[j]])',
                       robustSE=TRUE,
                       sims=1000, data=ukbb_cogloop)

        med2dep <- summary(lm(scale(as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~ 
                scale(ukbb_cogloop$WMH_logscaled) +
                scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop))

        Stats[index,][1]=colnames(ukbb_cogloop)[j]
        Stats[index,][2]=med.out$d0 # ACME estimate
        Stats[index,][3]=med.out$d0.p # ACME p-value
        Stats[index,][4]=med.out$z0 # ADE e
        Stats[index,][5]=med.out$z0.p # ADE p
        Stats[index,][6]=med.out$tau.coef # Total e
        Stats[index,][7]=med.out$tau.p # Total p
        Stats[index,][8]=med.out$n0 # Proportion mediated e
        Stats[index,][9]=med.out$n0.p # Proportion mediated p
        Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol


        index=index+1

        }      
    
    Stats_adj=Stats
    Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
    Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
    Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
    Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
    Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

    Stats_adj=as.data.frame(Stats_adj)
    colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')
    
    write.table(Stats_adj, paste('/dagher/dagher11/filip/OBAL/output/BMI_CT_',var_list[i],sep='') ,quote=FALSE, row.names=FALSE)
                                 
}

### Cortical volume

In [1254]:
var_list=c('maximum_digits_remembered_correctly_f4282_2_0',
          'fluid_intelligence_score_f20016_2_0',
          'Prospective_memory',
          'Mean_pairs')


for (i in 1:length(var_list)) {
    
    ukbb_cogloop=ukbb_cog[complete.cases(ukbb_cog[,grep(var_list[i], colnames(ukbb_cog))]),]
    
    Stats=matrix(nrow=length(c(89:119, 151:181)),ncol=10) # volume variables
    index=1

    for (j in c(89:119, 151:181)) { #All volume variables 

  
        ukbb_cogloop=ukbb_cogloop[complete.cases(ukbb_cogloop[,c(89:119, 151:181)]),]

        med.fit <- lm(scale(ukbb_cogloop[[j]]/ukbb_cogloop$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0) ~
                  scale(ukbb_cogloop$logBMI3) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)

        out.fit_cog=lm((as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~
                  scale(ukbb_cogloop[[j]]/ukbb_cogloop$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0) +
                  scale(ukbb_cogloop$logBMI3) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)


        med.out <- mediate(med.fit, out.fit_cog, treat='scale(ukbb_cogloop$logBMI3)',
                       mediator='scale(ukbb_cogloop[[j]]/ukbb_cogloop$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0)',
                       robustSE=TRUE,
                       sims=1000, data=ukbb_cogloop)

        med2dep <- summary(lm(scale(as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~ 
                scale(ukbb_cogloop$WMH_logscaled) +
                scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop))

        Stats[index,][1]=colnames(ukbb_cogloop)[j]
        Stats[index,][2]=med.out$d0 # ACME estimate
        Stats[index,][3]=med.out$d0.p # ACME p-value
        Stats[index,][4]=med.out$z0 # ADE e
        Stats[index,][5]=med.out$z0.p # ADE p
        Stats[index,][6]=med.out$tau.coef # Total e
        Stats[index,][7]=med.out$tau.p # Total p
        Stats[index,][8]=med.out$n0 # Proportion mediated e
        Stats[index,][9]=med.out$n0.p # Proportion mediated p
        Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol


        index=index+1

        }      
    
    Stats_adj=Stats
    Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
    Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
    Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
    Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
    Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

    Stats_adj=as.data.frame(Stats_adj)
    colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')
    
    write.table(Stats_adj, paste('/dagher/dagher11/filip/OBAL/output/BMI_VOL_',var_list[i],sep=''),quote=FALSE, row.names=FALSE)
                                 
}

### Subcortical volume

In [1277]:
var_list=c('maximum_digits_remembered_correctly_f4282_2_0',
          'fluid_intelligence_score_f20016_2_0',
          'Prospective_memory',
          'Mean_pairs')


for (i in 1:length(var_list)) {
    
    ukbb_cogloop=ukbb_cog[complete.cases(ukbb_cog[,grep(var_list[i], colnames(ukbb_cog))]),]
    
    Stats=matrix(nrow=length(c(2287:2300)),ncol=10) # volume variables
    index=1

  for (j in c(2287:2300)) { #All volume variables 

  
        ukbb_cogloop=ukbb_cogloop[complete.cases(ukbb_cogloop[,c(2287:2300)]),]
      
        ukbb_cogloop$submediator=scale(ukbb_cogloop[[j]]*ukbb_cogloop[[2116]]) #Necessary because mediator package complains when there is * character

        med.fit <- lm(ukbb_cogloop$submediator ~
                  scale(ukbb_cogloop$logBMI3) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)

        out.fit_cog=lm(as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]]) ~
                  ukbb_cogloop$submediator +
                  scale(ukbb_cogloop$logBMI3) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)


        med.out <- mediate(med.fit, out.fit_cog, treat='scale(ukbb_cogloop$logBMI3)',
                       mediator='ukbb_cogloop$submediator',
                       robustSE=TRUE,
                       sims=1000, data=ukbb_cogloop)

        med2dep <- summary(lm(scale(as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~ 
                scale(ukbb_cogloop$WMH_logscaled) +
                scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop))

        Stats[index,][1]=colnames(ukbb_cogloop)[j]
        Stats[index,][2]=med.out$d0 # ACME estimate
        Stats[index,][3]=med.out$d0.p # ACME p-value
        Stats[index,][4]=med.out$z0 # ADE e
        Stats[index,][5]=med.out$z0.p # ADE p
        Stats[index,][6]=med.out$tau.coef # Total e
        Stats[index,][7]=med.out$tau.p # Total p
        Stats[index,][8]=med.out$n0 # Proportion mediated e
        Stats[index,][9]=med.out$n0.p # Proportion mediated p
        Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol


        index=index+1

        }      
    
    Stats_adj=Stats
    Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
    Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
    Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
    Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
    Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

    Stats_adj=as.data.frame(Stats_adj)
    colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')
    
    write.table(Stats_adj, paste('/dagher/dagher11/filip/OBAL/output/BMI_SUBVOL_',var_list[i],sep=''),quote=FALSE, row.names=FALSE)
                                 
}

## WHR -> CT/VOL/SUBVOL -> Cognition

## CT

In [1278]:
var_list=c('maximum_digits_remembered_correctly_f4282_2_0',
          'fluid_intelligence_score_f20016_2_0',
          'Prospective_memory',
          'RT')


for (i in 1:length(var_list)) {
    
    ukbb_cogloop=ukbb_cog[complete.cases(ukbb_cog[,grep(var_list[i], colnames(ukbb_cog))]),]
    
    Stats=matrix(nrow=length(c(58:88, 120:150)),ncol=10) # CT variables
    index=1

    for (j in c(58:88, 120:150)) { #All thickness variables 58:88, 120:150
  
        ukbb_cogloop=ukbb_cogloop[complete.cases(ukbb_cogloop[,c(58:88, 120:150)]),]

        med.fit <- lm(scale(ukbb_cogloop[[j]]) ~
                  scale(ukbb_cogloop$WHR) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)

        out.fit_cog=lm((as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~
                  scale(ukbb_cogloop[[j]]) +
                  scale(ukbb_cogloop$WHR) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)


        med.out <- mediate(med.fit, out.fit_cog, treat='scale(ukbb_cogloop$WHR)',
                       mediator='scale(ukbb_cogloop[[j]])',
                       robustSE=TRUE,
                       sims=1000, data=ukbb_cogloop)

        med2dep <- summary(lm(scale(as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~ 
                scale(ukbb_cogloop$WMH_logscaled) +
                scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop))

        Stats[index,][1]=colnames(ukbb_cogloop)[j]
        Stats[index,][2]=med.out$d0 # ACME estimate
        Stats[index,][3]=med.out$d0.p # ACME p-value
        Stats[index,][4]=med.out$z0 # ADE e
        Stats[index,][5]=med.out$z0.p # ADE p
        Stats[index,][6]=med.out$tau.coef # Total e
        Stats[index,][7]=med.out$tau.p # Total p
        Stats[index,][8]=med.out$n0 # Proportion mediated e
        Stats[index,][9]=med.out$n0.p # Proportion mediated p
        Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol


        index=index+1

        }      
    
    Stats_adj=Stats
    Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
    Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
    Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
    Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
    Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

    Stats_adj=as.data.frame(Stats_adj)
    colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')
    
    write.table(Stats_adj, paste('/dagher/dagher11/filip/OBAL/output/WHR_CT_',var_list[i],sep=''),quote=FALSE, row.names=FALSE)
                                 
}

### Cortical volume

In [1279]:
var_list=c('maximum_digits_remembered_correctly_f4282_2_0',
          'fluid_intelligence_score_f20016_2_0',
          'Prospective_memory',
          'RT')


for (i in 1:length(var_list)) {
    
    ukbb_cogloop=ukbb_cog[complete.cases(ukbb_cog[,grep(var_list[i], colnames(ukbb_cog))]),]
    
    Stats=matrix(nrow=length(c(89:119, 151:181)),ncol=10) # CT variables
    index=1

    for (j in c(89:119, 151:181)) { #All volume variables 

  
        ukbb_cogloop=ukbb_cogloop[complete.cases(ukbb_cogloop[,c(89:119, 151:181)]),]

        med.fit <- lm(scale(ukbb_cogloop[[j]]/ukbb_cogloop$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0) ~
                  scale(ukbb_cogloop$WHR) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)

        out.fit_cog=lm((as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~
                  scale(ukbb_cogloop[[j]]/ukbb_cogloop$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0) +
                  scale(ukbb_cogloop$WHR) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)


        med.out <- mediate(med.fit, out.fit_cog, treat='scale(ukbb_cogloop$WHR)',
                       mediator='scale(ukbb_cogloop[[j]]/ukbb_cogloop$volume_of_estimatedtotalintracranial_whole_brain_f26521_2_0)',
                       robustSE=TRUE,
                       sims=1000, data=ukbb_cogloop)

        med2dep <- summary(lm(scale(as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~ 
                scale(ukbb_cogloop$WMH_logscaled) +
                scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop))

        Stats[index,][1]=colnames(ukbb_cogloop)[j]
        Stats[index,][2]=med.out$d0 # ACME estimate
        Stats[index,][3]=med.out$d0.p # ACME p-value
        Stats[index,][4]=med.out$z0 # ADE e
        Stats[index,][5]=med.out$z0.p # ADE p
        Stats[index,][6]=med.out$tau.coef # Total e
        Stats[index,][7]=med.out$tau.p # Total p
        Stats[index,][8]=med.out$n0 # Proportion mediated e
        Stats[index,][9]=med.out$n0.p # Proportion mediated p
        Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol


        index=index+1

        }      
    
    Stats_adj=Stats
    Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
    Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
    Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
    Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
    Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

    Stats_adj=as.data.frame(Stats_adj)
    colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')
    
    write.table(Stats_adj, paste('/dagher/dagher11/filip/OBAL/output/WHR_VOL_',var_list[i],sep=''),quote=FALSE, row.names=FALSE)
                                 
}

## Subcortical volume

In [1280]:
var_list=c('maximum_digits_remembered_correctly_f4282_2_0',
          'fluid_intelligence_score_f20016_2_0',
          'Prospective_memory',
          'RT')


for (i in 1:length(var_list)) {
    
    ukbb_cogloop=ukbb_cog[complete.cases(ukbb_cog[,grep(var_list[i], colnames(ukbb_cog))]),]
    
    Stats=matrix(nrow=length(c(2287:2300)),ncol=10) # CT variables
    index=1

  for (j in c(2287:2300)) { #All volume variables 

  
        ukbb_cogloop=ukbb_cogloop[complete.cases(ukbb_cogloop[,c(2287:2300)]),]
      
        ukbb_cogloop$submediator=scale(ukbb_cogloop[[j]]*ukbb_cogloop[[2116]]) #Necessary because mediator package complains when there is * character


        med.fit <- lm(ukbb_cogloop$submediator ~
                  scale(ukbb_cogloop$WHR) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)

        out.fit_cog=lm((as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~
                  ukbb_cogloop$submediator +
                  scale(ukbb_cogloop$WHR) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)


        med.out <- mediate(med.fit, out.fit_cog, treat='scale(ukbb_cogloop$WHR)',
                       mediator='ukbb_cogloop$submediator',
                       robustSE=TRUE,
                       sims=1000, data=ukbb_cogloop)

        med2dep <- summary(lm(scale(as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~ 
                scale(ukbb_cogloop$WMH_logscaled) +
                scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop))

        Stats[index,][1]=colnames(ukbb_cogloop)[j]
        Stats[index,][2]=med.out$d0 # ACME estimate
        Stats[index,][3]=med.out$d0.p # ACME p-value
        Stats[index,][4]=med.out$z0 # ADE e
        Stats[index,][5]=med.out$z0.p # ADE p
        Stats[index,][6]=med.out$tau.coef # Total e
        Stats[index,][7]=med.out$tau.p # Total p
        Stats[index,][8]=med.out$n0 # Proportion mediated e
        Stats[index,][9]=med.out$n0.p # Proportion mediated p
        Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol


        index=index+1

        }      
    
    Stats_adj=Stats
    Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
    Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
    Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
    Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
    Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

    Stats_adj=as.data.frame(Stats_adj)
    colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')
    
    write.table(Stats_adj, paste('/dagher/dagher11/filip/OBAL/output/WHR_SUBVOL_',var_list[i],sep=''),quote=FALSE, row.names=FALSE)
                                 
}

## Investigate significant results for BMI/WHR -> CT/VOL/SUBVOL -> Cog
None of them are significant

In [1358]:
var_list=c('maximum_digits_remembered_correctly_f4282_2_0',
          'fluid_intelligence_score_f20016_2_0',
          'Prospective_memory',
          'Mean_pairs',
          'RT')

measures_list=c('BMI_CT','BMI_VOL','BMI_SUBVOL','BMI_area','WHR_CT','WHR_VOL','WHR_SUBVOL','WHR_area')

for (i in 1:length(measures_list)) {
    for (j in 1:length(var_list)) {
        Stats_table=tryCatch(read.table(paste('/dagher/dagher11/filip/OBAL/output/',measures_list[i],'_',var_list[j],
                                     sep=''), header=TRUE, sep=' '), error = function(e) e) # Use tryCatch: if read.table returns an error then Stats_table is of class error
        if(inherits(Stats_table, "error")) next # if Stats_table is of class error then skips the loop
        print(paste(measures_list[i],'_',var_list[j], sep=''))
        temp=Stats_table[Stats_table[3]<0.05 & Stats_table[7]<0.05 & Stats_table[10]<0.05 & Stats_table[6]<0,]
        temp
    }   
}

# Surface area : BMI, VAT, WHR -> WMH -> surface area

# BMI

In [1287]:
set.seed(2014)
med.fit <- lm(scale(ukbb_area$WMH_logscaled) ~
          scale(ukbb_area$logBMI3) +
          scale(ukbb_area$age_when_attended_assessment_centre_f21003_2_0) +
          ukbb_area$sex_f31_0_0, data=ukbb_area)

In [1288]:
Stats=matrix(nrow=length(c(grep('^area_of.*_2_',colnames(ukbb_area)))),ncol=10)
index=1

for (i in c(grep('^area_of.*_2_',colnames(ukbb_area)))) { #All area variables 
  
    out.fit <- lm(scale(ukbb_area[[i]]) ~ 
                scale(ukbb_area$WMH_logscaled) +
                scale(ukbb_area$logBMI3) +
                scale(ukbb_area$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_area$sex_f31_0_0, data=ukbb_area)
    
    med.out <- mediate(med.fit, out.fit, treat='scale(ukbb_area$logBMI3)',
                   mediator='scale(ukbb_area$WMH_logscaled)',
                   robustSE=TRUE,
                   sims=1000, data=ukbb_area)
    
    med2dep <- summary(lm(scale(ukbb_area[[i]]) ~ 
                scale(ukbb_area$WMH_logscaled) +
                scale(ukbb_area$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_area$sex_f31_0_0, data=ukbb_area)) 
      
    
    Stats[index,][1]=colnames(ukbb_area)[i]
    Stats[index,][2]=med.out$d0 # ACME estimate
    Stats[index,][3]=med.out$d0.p # ACME p-value
    Stats[index,][4]=med.out$z0 # ADE e
    Stats[index,][5]=med.out$z0.p # ADE p
    Stats[index,][6]=med.out$tau.coef # Total e
    Stats[index,][7]=med.out$tau.p # Total p
    Stats[index,][8]=med.out$n0 # Proportion mediated e
    Stats[index,][9]=med.out$n0.p # Proportion mediated p
    Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> CT
       
     
    index=index+1
}
Stats_adj=Stats
Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

In [1289]:
write.table(Stats_adj[,1:10],'/dagher/dagher11/filip/OBAL/output/WMH_area_mediation.csv',sep=',',quote=FALSE, row.names=FALSE)

# WHR

In [1290]:
set.seed(2014)
med.fit <- lm(scale(ukbb_area$WMH_logscaled) ~
          scale(ukbb_area$WHR) +
          scale(ukbb_area$age_when_attended_assessment_centre_f21003_2_0) +
          ukbb_area$sex_f31_0_0, data=ukbb_area)

In [1291]:
Stats=matrix(nrow=length(c(grep('^area_of.*_2_',colnames(ukbb_area)))),ncol=10)
index=1

for (i in c(grep('^area_of.*_2_',colnames(ukbb_area)))) { #All area variables 
  
    out.fit <- lm(scale(ukbb_area[[i]]) ~ 
                scale(ukbb_area$WMH_logscaled) +
                scale(ukbb_area$WHR) +
                scale(ukbb_area$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_area$sex_f31_0_0, data=ukbb_area)
    
    med.out <- mediate(med.fit, out.fit, treat='scale(ukbb_area$WHR)',
                   mediator='scale(ukbb_area$WMH_logscaled)',
                   robustSE=TRUE,
                   sims=1000, data=ukbb_area)
    
    med2dep <- summary(lm(scale(ukbb_area[[i]]) ~ 
                scale(ukbb_area$WMH_logscaled) +
                scale(ukbb_area$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_area$sex_f31_0_0, data=ukbb_area)) 
      
    
    Stats[index,][1]=colnames(ukbb_area)[i]
    Stats[index,][2]=med.out$d0 # ACME estimate
    Stats[index,][3]=med.out$d0.p # ACME p-value
    Stats[index,][4]=med.out$z0 # ADE e
    Stats[index,][5]=med.out$z0.p # ADE p
    Stats[index,][6]=med.out$tau.coef # Total e
    Stats[index,][7]=med.out$tau.p # Total p
    Stats[index,][8]=med.out$n0 # Proportion mediated e
    Stats[index,][9]=med.out$n0.p # Proportion mediated p
    Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> CT
       
     
    index=index+1
}
Stats_adj=Stats
Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

In [1292]:
write.table(Stats_adj[,1:10],'/dagher/dagher11/filip/OBAL/output/WMH_area_mediation_WHR.csv',sep=',',quote=FALSE, row.names=FALSE)

## VAT
NO SIGNIFICANT MEDIATION

In [1293]:
set.seed(2014)
med.fit <- lm(scale(ukbb_area$WMH_logscaled) ~
          scale(log(ukbb_area[[1990]]/(ukbb_area[[1993]]))) +
          scale(ukbb_area$age_when_attended_assessment_centre_f21003_2_0) +
          ukbb_area$sex_f31_0_0, data=ukbb_area)

In [1294]:
Stats=matrix(nrow=length(c(grep('^area_of.*_2_',colnames(ukbb_area)))),ncol=10)
index=1

for (i in c(grep('^area_of.*_2_',colnames(ukbb_area)))) { #All area variables 
  
    out.fit <- lm(scale(ukbb_area[[i]]) ~ 
                scale(ukbb_area$WMH_logscaled) +
                scale(log(ukbb_area[[1990]]/(ukbb_area[[1993]]))) + +
                scale(ukbb_area$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_area$sex_f31_0_0, data=ukbb_area)
    
    med.out <- mediate(med.fit, out.fit, treat='scale(log(ukbb_area[[1990]]/(ukbb_area[[1993]])))',
                   mediator='scale(ukbb_area$WMH_logscaled)',
                   robustSE=TRUE,
                   sims=1000, data=ukbb_area)
    
    med2dep <- summary(lm(scale(ukbb_area[[i]]) ~ 
                scale(ukbb_area$WMH_logscaled) +
                scale(ukbb_area$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_area$sex_f31_0_0, data=ukbb_area)) 
      
    
    Stats[index,][1]=colnames(ukbb_area)[i]
    Stats[index,][2]=med.out$d0 # ACME estimate
    Stats[index,][3]=med.out$d0.p # ACME p-value
    Stats[index,][4]=med.out$z0 # ADE e
    Stats[index,][5]=med.out$z0.p # ADE p
    Stats[index,][6]=med.out$tau.coef # Total e
    Stats[index,][7]=med.out$tau.p # Total p
    Stats[index,][8]=med.out$n0 # Proportion mediated e
    Stats[index,][9]=med.out$n0.p # Proportion mediated p
    Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> CT
       
     
    index=index+1
}
Stats_adj=Stats
Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

Stats_adj=as.data.frame(Stats_adj)
colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')

In [1295]:
write.table(Stats_adj[,1:10],'/dagher/dagher11/filip/OBAL/output/WMH_area_mediation_VAT.csv',sep=',',quote=FALSE, row.names=FALSE)

## Mediation BMI/WHR -> surface area -> cognition  
#### Not for VAT as it is not related to cognition

## BMI

In [1355]:
var_list=c('maximum_digits_remembered_correctly_f4282_2_0',
          'fluid_intelligence_score_f20016_2_0',
          'Prospective_memory',
          'Mean_pairs')


for (i in 1:length(var_list)) {
    
    ukbb_cogloop=ukbb_cog[complete.cases(ukbb_cog[,grep(var_list[i], colnames(ukbb_cog))]),]
    
    Stats=matrix(nrow=length(c(grep('^area_of.*_2_',colnames(ukbb_area)))),ncol=10) # CT variables
    index=1

    for (j in c(grep('^area_of.*_2_',colnames(ukbb_area)))) { #All thickness variables 58:88, 120:150
        
        ukbb_cogloop=ukbb_cogloop[complete.cases(ukbb_cogloop[,c(grep('^area_of.*_2_',colnames(ukbb_cogloop)))]),]

        med.fit <- lm(scale(ukbb_cogloop[[j]]) ~
                  scale(ukbb_cogloop$logBMI3) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)

        out.fit_cog=lm((as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~
                  scale(ukbb_cogloop[[j]]) +
                  scale(ukbb_cogloop$logBMI3) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)


        med.out <- mediate(med.fit, out.fit_cog, treat='scale(ukbb_cogloop$logBMI3)',
                       mediator='scale(ukbb_cogloop[[j]])',
                       robustSE=TRUE,
                       sims=1000, data=ukbb_cogloop)

        med2dep <- summary(lm(scale(as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~ 
                scale(ukbb_cogloop$WMH_logscaled) +
                scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop))

        Stats[index,][1]=colnames(ukbb_cogloop)[j]
        Stats[index,][2]=med.out$d0 # ACME estimate
        Stats[index,][3]=med.out$d0.p # ACME p-value
        Stats[index,][4]=med.out$z0 # ADE e
        Stats[index,][5]=med.out$z0.p # ADE p
        Stats[index,][6]=med.out$tau.coef # Total e
        Stats[index,][7]=med.out$tau.p # Total p
        Stats[index,][8]=med.out$n0 # Proportion mediated e
        Stats[index,][9]=med.out$n0.p # Proportion mediated p
        Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol


        index=index+1

        }      
    
    Stats_adj=Stats
    Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
    Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
    Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
    Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
    Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

    Stats_adj=as.data.frame(Stats_adj)
    colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')
    
    write.table(Stats_adj, paste('/dagher/dagher11/filip/OBAL/output/BMI_area_',var_list[i],sep='') ,quote=FALSE, row.names=FALSE)
                                 
}

## WHR

In [1357]:
var_list=c('maximum_digits_remembered_correctly_f4282_2_0',
          'fluid_intelligence_score_f20016_2_0',
          'Prospective_memory',
          'RT')


for (i in 1:length(var_list)) {
    
    ukbb_cogloop=ukbb_cog[complete.cases(ukbb_cog[,grep(var_list[i], colnames(ukbb_cog))]),]
    
    Stats=matrix(nrow=length(c(grep('^area_of.*_2_',colnames(ukbb_area)))),ncol=10) # CT variables
    index=1

    for (j in c(grep('^area_of.*_2_',colnames(ukbb_area)))) { #All thickness variables 58:88, 120:150
  
        ukbb_cogloop=ukbb_cogloop[complete.cases(ukbb_cogloop[,c(grep('^area_of.*_2_',colnames(ukbb_cogloop)))]),]

        med.fit <- lm(scale(ukbb_cogloop[[j]]) ~
                  scale(ukbb_cogloop$WHR) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)

        out.fit_cog=lm((as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~
                  scale(ukbb_cogloop[[j]]) +
                  scale(ukbb_cogloop$WHR) +
                  scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                  ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop)


        med.out <- mediate(med.fit, out.fit_cog, treat='scale(ukbb_cogloop$WHR)',
                       mediator='scale(ukbb_cogloop[[j]])',
                       robustSE=TRUE,
                       sims=1000, data=ukbb_cogloop)

        med2dep <- summary(lm(scale(as.numeric(ukbb_cogloop[[grep(var_list[i], colnames(ukbb_cogloop))]])) ~ 
                scale(ukbb_cogloop$WMH_logscaled) +
                scale(ukbb_cogloop$age_when_attended_assessment_centre_f21003_2_0) +
                ukbb_cogloop$sex_f31_0_0, data=ukbb_cogloop))

        Stats[index,][1]=colnames(ukbb_cogloop)[j]
        Stats[index,][2]=med.out$d0 # ACME estimate
        Stats[index,][3]=med.out$d0.p # ACME p-value
        Stats[index,][4]=med.out$z0 # ADE e
        Stats[index,][5]=med.out$z0.p # ADE p
        Stats[index,][6]=med.out$tau.coef # Total e
        Stats[index,][7]=med.out$tau.p # Total p
        Stats[index,][8]=med.out$n0 # Proportion mediated e
        Stats[index,][9]=med.out$n0.p # Proportion mediated p
        Stats[index,][10]=med2dep$coefficients[2,][4] # p-value of WMH -> Vol


        index=index+1

        }      
    
    Stats_adj=Stats
    Stats_adj[,3]=p.adjust(Stats_adj[,3], method = 'BH')
    Stats_adj[,5]=p.adjust(Stats_adj[,5], method = 'BH')
    Stats_adj[,7]=p.adjust(Stats_adj[,7], method = 'BH')
    Stats_adj[,9]=p.adjust(Stats_adj[,9], method = 'BH')
    Stats_adj[,10]=p.adjust(Stats_adj[,10], method = 'BH')

    Stats_adj=as.data.frame(Stats_adj)
    colnames(Stats_adj)=c('Parcel','ACMEest','ACMEp','ADEest','ADEp','Totalest','Totalp','Proportionest','Proportionp','Med2Depp')
    
    write.table(Stats_adj, paste('/dagher/dagher11/filip/OBAL/output/WHR_area_',var_list[i],sep=''),quote=FALSE, row.names=FALSE)
                                 
}

## Check correlations of WMH with FA and MD measures

In [1524]:
Stats_corr=matrix(ncol=3,nrow=length(c(5123:5218)))
index=1
for (i in c(5123:5218)) {
    
    a=summary(lm(ukbb_WM$WMH_logscaled ~
        scale(ukbb_WM[[i]]) +
        scale(ukbb_WM$age_when_attended_assessment_centre_f21003_2_0) +
        ukbb_WM$sex_f31_0_0))
    
    Stats_corr[index,][1]=colnames(ukbb_WM)[i]
    Stats_corr[index,][2]=a$coefficients[2,][4]
    Stats_corr[index,][3]=a$coefficients[2,][3]   
    
    index=index+1
    
}

In [1525]:
print(Stats_corr[p.adjust(Stats_corr[,2], method='BH')<0.05])

In [1526]:
write.table(Stats_corr,'/dagher/dagher11/filip/OBAL/output/WMH_FA.csv',sep=',')

In [1527]:
Stats_corr=matrix(ncol=3,nrow=length(c(5123:5218)))
index=1
for (i in c(5123:5218)) {
    
    a=summary(lm(ukbb_WM$logBMI3 ~
        scale(ukbb_WM[[i]]) +
        scale(ukbb_WM$age_when_attended_assessment_centre_f21003_2_0) +
        ukbb_WM$sex_f31_0_0))
    
    Stats_corr[index,][1]=colnames(ukbb_WM)[i]
    Stats_corr[index,][2]=a$coefficients[2,][4]
    Stats_corr[index,][3]=a$coefficients[2,][3]   
    
    index=index+1
    
}

In [1528]:
print(Stats_corr[p.adjust(Stats_corr[,2], method='BH')<0.05])

In [1529]:
write.table(Stats_corr,'/dagher/dagher11/filip/OBAL/output/BMI_FA.csv',sep=',')

In [1530]:
Stats_corr=matrix(ncol=3,nrow=length(c(5123:5218)))
index=1
for (i in c(5123:5218)) {
    
    a=summary(lm(ukbb_WM$WHR ~
        scale(ukbb_WM[[i]]) +
        scale(ukbb_WM$age_when_attended_assessment_centre_f21003_2_0) +
        ukbb_WM$sex_f31_0_0))
    
    Stats_corr[index,][1]=colnames(ukbb_WM)[i]
    Stats_corr[index,][2]=a$coefficients[2,][4]
    Stats_corr[index,][3]=a$coefficients[2,][3]   
    
    index=index+1
    
}

In [1531]:
print(Stats_corr[p.adjust(Stats_corr[,2], method='BH')<0.05])

In [1532]:
write.table(Stats_corr,'/dagher/dagher11/filip/OBAL/output/WHR_FA.csv',sep=',')

In [1533]:
colnames(ukbb_WM)[c(5123:5218)]

In [1534]:
Stats_corr=matrix(ncol=3,nrow=length(c(5123:5218)))
index=1
for (i in c(5123:5218)) {
    
    a=summary(lm(scale(log(ukbb_WM[[1990]]/(ukbb_WM[[1993]]))) ~
        scale(ukbb_WM[[i]]) +
        scale(ukbb_WM$age_when_attended_assessment_centre_f21003_2_0) +
        ukbb_WM$sex_f31_0_0))
    
    Stats_corr[index,][1]=colnames(ukbb_WM)[i]
    Stats_corr[index,][2]=a$coefficients[2,][4]
    Stats_corr[index,][3]=a$coefficients[2,][3]   
    
    index=index+1
    
}

In [1535]:
print(Stats_corr[p.adjust(Stats_corr[,2], method='BH')<0.05])

In [1536]:
write.table(Stats_corr,'/dagher/dagher11/filip/OBAL/output/VAT_FA.csv',sep=',')